In [122]:
import pandas as pd
from pyprojroot.here import here

In [123]:
nfi_data_raw = pd.read_csv(
    "./20231108-211512_nfi_dataset_for_analysis copy.csv", index_col=0
)
nfi_data_raw

/var/folders/39/534x_msj7sl1kcxj2mg109x40000gn/T/ipykernel_3305/125098809.py:1: DtypeWarning: Columns (51,58,61,73,74,75,88,89,90,91,106,107,112,113,115,119,125,127,156) have mixed types. Specify dtype option on import or set low_memory=False.
  nfi_data_raw = pd.read_csv(


,idp,perc_cut,dominant_species,dominant_tree_class,a,tree_id,visit_1,visit_2,revisit_state,hrb,...,obsriv2,typriv,distriv,denivriv,lign2,lign1,herb,mousse,obsveget,obsdate
1,632691,0.0,none,pinus,3,632691_3,2011,2016,revisited,NaN,...,NaN,NaN,NaN,NaN,10.0,1.0,8.0,2.0,NaN,NaN
2,632691,0.0,none,pinus,1,632691_1,2011,2016,revisited,NaN,...,NaN,NaN,NaN,NaN,10.0,1.0,8.0,2.0,NaN,NaN
3,632691,0.0,none,pinus,2,632691_2,2011,2016,revisited,NaN,...,NaN,NaN,NaN,NaN,10.0,1.0,8.0,2.0,NaN,NaN
4,632691,0.0,none,pinus,4,632691_4,2011,2016,revisited,NaN,...,NaN,NaN,NaN,NaN,10.0,1.0,8.0,2.0,NaN,NaN
5,632691,0.0,none,pinus,5,632691_5,2011,2016,revisited,NaN,...,NaN,NaN,NaN,NaN,10.0,1.0,8.0,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375268,1131437,0.0,Quercus,broadleaf,15,1131437_15,2016,2021,revisited,NaN,...,NaN,NaN,NaN,NaN,10.0,1.0,0.0,0.0,NaN,NaN
375269,1131437,0.0,Quercus,broadleaf,16,1131437_16,2016,2021,revisited,NaN,...,NaN,NaN,NaN,NaN,10.0,1.0,0.0,0.0,NaN,NaN
375270,1131437,0.0,Quercus,broadleaf,17,1131437_17,2016,2021,revisited,NaN,...,NaN,NaN,NaN,NaN,10.0,1.0,0.0,0.0,NaN,NaN
375271,1131437,0.0,Quercus,broadleaf,18,1131437_18,2016,2021,revisited,NaN,...,NaN,NaN,NaN,NaN,10.0,1.0,0.0,0.0,NaN,NaN


In [159]:
nfi_data = (
    nfi_data_raw[["idp", "visit_1", "lon", "lat"]]
    .rename({"visit_1": "first_year", "lon": "x", "lat": "y"}, axis=1)
    .drop_duplicates()
    .reset_index(drop=True)
)

# Create data with idp: For later merging of nfi data and predictor data
nfi_data_with_idp = (
    nfi_data.reset_index(drop=False)
    .rename(columns={"index": "SiteID"})
    .assign(SiteID=lambda x: x["SiteID"] + 1)
)

# Fix order
nfi_data_with_idp = nfi_data_with_idp[["first_year", "SiteID", "x", "y", "idp"]]
nfi_data_with_idp.to_csv("nfi_final_sites_with_idp.csv", index=False)
# nfi_data_with_idp

In [158]:
# Create data without idp: For faster extraction of predictor data
nfi_data_without_idp = (
    nfi_data.drop(columns="idp")
    .drop_duplicates()
    .reset_index(drop=False)
    .rename(columns={"index": "SiteID"})
    .assign(SiteID=lambda x: x["SiteID"] + 1)
)

# Fix order
nfi_data_without_idp = nfi_data_without_idp[["first_year", "SiteID", "x", "y"]]
nfi_data_without_idp.to_csv("nfi_final_sites_without_idp.csv", index=False)

In [160]:
pd.read_csv("nfi_final_sites_with_idp.csv")

,first_year,SiteID,x,y,idp
0,2011,1,-2.842824,48.337505,632691
1,2012,2,3.349757,46.198025,702597
2,2012,3,3.361577,46.827747,706240
3,2012,4,0.402182,48.201563,708321
4,2012,5,6.461081,43.281648,708369
...,...,...,...,...,...
38691,2016,38692,2.156438,42.686748,1131409
38692,2016,38693,7.424664,47.899971,1131410
38693,2016,38694,-3.221877,48.851908,1131419
38694,2016,38695,3.757202,44.924973,1131424


In [161]:
pd.read_csv("nfi_final_sites_without_idp.csv")

,first_year,SiteID,x,y
0,2011,1,-2.842824,48.337505
1,2012,2,3.349757,46.198025
2,2012,3,3.361577,46.827747
3,2012,4,0.402182,48.201563
4,2012,5,6.461081,43.281648
...,...,...,...,...
38021,2016,38692,2.156438,42.686748
38022,2016,38693,7.424664,47.899971
38023,2016,38694,-3.221877,48.851908
38024,2016,38695,3.757202,44.924973


---
Code below was to fix old formatting structure of the data. Do not rerun

In [162]:
# old_data = pd.read_csv(here("python/01_download_raw_gee_data/sites_years.csv"))
# old_data = old_data.rename(columns={"first_visit": "first_year", "id": "SiteID"})
# # old_data

In [163]:
# old_data = old_data.merge(
#     nfi_data[["x", "y", "first_year", "idp"]].drop_duplicates(),
#     on=["x", "y", "first_year"],
#     how="left",
# )
# old_data

,first_year,SiteID,x,y,idp
0,2011,1,-2.842824,48.337505,632691.0
1,2012,2,3.349757,46.198025,702597.0
2,2012,3,3.361577,46.827747,706240.0
3,2012,4,0.402182,48.201563,708321.0
4,2012,5,6.461081,43.281648,708369.0
...,...,...,...,...,...
38967,2016,38298,2.156438,42.686748,1131409.0
38968,2016,38299,7.424664,47.899971,1131410.0
38969,2016,38300,-3.221877,48.851908,1131419.0
38970,2016,38301,3.757202,44.924973,1131424.0


In [164]:
# # Save data with idp attached to it for later merging of predictor data and nfi data
# old_data.to_csv("nfi_final_sites_with_idp.csv", index=False)

# # Save data without idp attached for faster download and extraction of predictor data
# old_data[["first_year", "SiteID", "x", "y"]].drop_duplicates().to_csv(
#     "nfi_final_sites_without_idp.csv", index=False
# )

In [165]:
# pd.read_csv("nfi_final_sites_with_idp.csv")

,first_year,SiteID,x,y,idp
0,2011,1,-2.842824,48.337505,632691.0
1,2012,2,3.349757,46.198025,702597.0
2,2012,3,3.361577,46.827747,706240.0
3,2012,4,0.402182,48.201563,708321.0
4,2012,5,6.461081,43.281648,708369.0
...,...,...,...,...,...
38967,2016,38298,2.156438,42.686748,1131409.0
38968,2016,38299,7.424664,47.899971,1131410.0
38969,2016,38300,-3.221877,48.851908,1131419.0
38970,2016,38301,3.757202,44.924973,1131424.0


In [166]:
# pd.read_csv("nfi_final_sites_without_idp.csv")

,first_year,SiteID,x,y
0,2011,1,-2.842824,48.337505
1,2012,2,3.349757,46.198025
2,2012,3,3.361577,46.827747
3,2012,4,0.402182,48.201563
4,2012,5,6.461081,43.281648
...,...,...,...,...
38297,2016,38298,2.156438,42.686748
38298,2016,38299,7.424664,47.899971
38299,2016,38300,-3.221877,48.851908
38300,2016,38301,3.757202,44.924973
